In [122]:
import librosa 
def extract_features(file_name):
   
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None 
     
    return mfccsscaled

In [123]:
# Load various imports 
import pandas as pd
import os
import librosa

# Set the path to the full UrbanSound dataset 
fulldatasetpath = '../dataset/audio/'

#metadata = pd.read_csv('../dataset/dataset.csv')
metadata = pd.read_csv('../dataset/datasetM2.csv')

features = []

# Iterate through each sound file and extract the features 
for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath(fulldatasetpath),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    
    class_label = row["class_name"]
    data = extract_features(file_name)
    
    features.append([data, class_label])

# Convert into a Panda dataframe 
featuresdf = pd.DataFrame(features, columns=['feature','class_label'])

print('Finished feature extraction from ', len(featuresdf), ' files') 

Finished feature extraction from  818  files


In [124]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# Convert features and corresponding classification labels into numpy arrays
X = np.array(featuresdf.feature.tolist())
y = np.array(featuresdf.class_label.tolist())

# Encode the classification labels
le = LabelEncoder()
yy = to_categorical(le.fit_transform(y)) 

In [125]:
# split the dataset 
from sklearn.model_selection import train_test_split 

x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 42)

In [126]:
### store the preprocessed data for use in the next notebook

%store x_train 
%store x_test 
%store y_train 
%store y_test 
%store yy 
%store le

Stored 'x_train' (ndarray)
Stored 'x_test' (ndarray)
Stored 'y_train' (ndarray)
Stored 'y_test' (ndarray)
Stored 'yy' (ndarray)
Stored 'le' (LabelEncoder)


In [127]:
# retrieve the preprocessed data from previous notebook

%store -r x_train 
%store -r x_test 
%store -r y_train 
%store -r y_test 
%store -r yy 
%store -r le

In [128]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics 

num_labels = yy.shape[1]
filter_size = 2

# Construct model 
model = Sequential()

model.add(Dense(256, input_shape=(40,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [129]:
# Compile the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam') 

In [130]:
# Display model architecture summary 
model.summary()

# Calculate pre-training accuracy 
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

print("Pre-training accuracy: %.4f%%" % accuracy)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 256)               10496     
_________________________________________________________________
activation_25 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 256)               65792     
_________________________________________________________________
activation_26 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 5)                

In [131]:
from keras.callbacks import ModelCheckpoint 
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.basic_mlp.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Train on 654 samples, validate on 164 samples
Epoch 1/100
654/654 [==============================] - 0s 352us/step - loss: 48.1164 - accuracy: 0.2187 - val_loss: 3.1033 - val_accuracy: 0.3537

Epoch 00001: val_loss improved from inf to 3.10330, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 2/100
654/654 [==============================] - 0s 138us/step - loss: 27.6231 - accuracy: 0.2095 - val_loss: 5.4773 - val_accuracy: 0.2378

Epoch 00002: val_loss did not improve from 3.10330
Epoch 3/100
654/654 [==============================] - 0s 140us/step - loss: 16.8139 - accuracy: 0.2125 - val_loss: 2.0876 - val_accuracy: 0.1585

Epoch 00003: val_loss improved from 3.10330 to 2.08762, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 4/100
654/654 [==============================] - 0s 141us/step - loss: 8.9123 - accuracy: 0.2355 - val_loss: 1.5684 - val_accuracy: 0.3110

Epoch 00004: val_loss improved from 2.08762 to 1.56843, saving model to saved_models/weights.b


Epoch 00040: val_loss did not improve from 1.53677
Epoch 41/100
654/654 [==============================] - 0s 235us/step - loss: 1.6092 - accuracy: 0.2752 - val_loss: 1.5424 - val_accuracy: 0.3537

Epoch 00041: val_loss did not improve from 1.53677
Epoch 42/100
654/654 [==============================] - 0s 185us/step - loss: 1.5882 - accuracy: 0.2676 - val_loss: 1.5613 - val_accuracy: 0.3537

Epoch 00042: val_loss did not improve from 1.53677
Epoch 43/100
654/654 [==============================] - 0s 173us/step - loss: 1.5830 - accuracy: 0.2844 - val_loss: 1.5247 - val_accuracy: 0.3537

Epoch 00043: val_loss improved from 1.53677 to 1.52475, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 44/100
654/654 [==============================] - 0s 162us/step - loss: 1.5747 - accuracy: 0.2859 - val_loss: 1.5374 - val_accuracy: 0.3537

Epoch 00044: val_loss did not improve from 1.52475
Epoch 45/100
654/654 [==============================] - 0s 140us/step - loss: 1.5880 - accurac

654/654 [==============================] - 0s 162us/step - loss: 1.4449 - accuracy: 0.3547 - val_loss: 1.2689 - val_accuracy: 0.5488

Epoch 00077: val_loss improved from 1.27860 to 1.26893, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 78/100
654/654 [==============================] - 0s 162us/step - loss: 1.4590 - accuracy: 0.3486 - val_loss: 1.2450 - val_accuracy: 0.5488

Epoch 00078: val_loss improved from 1.26893 to 1.24496, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 79/100
654/654 [==============================] - 0s 144us/step - loss: 1.4516 - accuracy: 0.3532 - val_loss: 1.2574 - val_accuracy: 0.5488

Epoch 00079: val_loss did not improve from 1.24496
Epoch 80/100
654/654 [==============================] - 0s 156us/step - loss: 1.4133 - accuracy: 0.3716 - val_loss: 1.2148 - val_accuracy: 0.5488

Epoch 00080: val_loss improved from 1.24496 to 1.21485, saving model to saved_models/weights.best.basic_mlp.hdf5
Epoch 81/100
654/654 [=============

In [132]:
# Evaluating the model on the training and testing set
score = model.evaluate(x_train, y_train, verbose=0)
print("Training Accuracy: ", score[1])

score = model.evaluate(x_test, y_test, verbose=0)
print("Testing Accuracy: ", score[1])

Training Accuracy:  0.4847094714641571
Testing Accuracy:  0.5487805008888245


In [133]:
import librosa 
import numpy as np 

def extract_feature(file_name):
   
    try:
        audio_data, sample_rate = librosa.load(file_name, res_type='kaiser_fast') 
        mfccs = librosa.feature.mfcc(y=audio_data, sr=sample_rate, n_mfcc=40)
        mfccsscaled = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file)
        return None, None

    return np.array([mfccsscaled])


In [134]:
def print_prediction(file_name):
    prediction_feature = extract_feature(file_name) 

    predicted_vector = model.predict_classes(prediction_feature)
    predicted_class = le.inverse_transform(predicted_vector) 
    print("The predicted class is:", predicted_class[0], '\n') 

    predicted_proba_vector = model.predict_proba(prediction_feature) 
    predicted_proba = predicted_proba_vector[0]
    for i in range(len(predicted_proba)): 
        category = le.inverse_transform(np.array([i]))
        print(category[0], "\t\t : ", format(predicted_proba[i], '.32f') )

In [135]:
# test

filename = '../dataset/audio/ebtest.wav' 
print_prediction(filename) 

The predicted class is: tap 

double_tap 		 :  0.41091313958168029785156250000000
long_tap 		 :  0.01297318190336227416992187500000
swipe 		 :  0.07753206789493560791015625000000
tap 		 :  0.43297278881072998046875000000000
zoom 		 :  0.06560879945755004882812500000000
